# Installation
You can create a conda environment to install all the dependencies:
```
conda env create -f environment.yml
```
After conda creates the environment, you can activate it with
```
conda activate valence-env
```
You can add valence-env as a Jupyter kernel:
```
python -m ipykernel install --user --name valence-env
```
After this step, valence-env should be available in Jupyter Notebook Kernel/Change Kernel menu.

In [62]:
%load_ext autoreload
%autoreload 2
#%matplotlib nbagg
import vtools.iotools as io
import vtools.obtools as ob
import openbabel
import vtools.vtools as vt
#import numpy as np
import matplotlib.pyplot as plt
from subprocess import Popen
from ipywidgets import interact, interactive
from ipywidgets import fixed, interact_manual 
from ipywidgets import Textarea, widgets, Layout, Accordion
from ipywidgets import VBox, HBox, Box, Text, BoundedIntText
import numpy as np
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
@interact(mol=Textarea(value='O',layout={'height': '100px','width':'400px'}),style=['line','cross','sphere','stick','cartoon'],radius=0.5)
def display_molecule(mol,style='stick',radius=0.7):
    import py3Dmol
    #http://3dmol.csb.pitt.edu/doc/types.html#AtomStyleSpec
    mol = ob.get_mol((mol),make3D=True)
    xyz = ob.get_xyz(mol)
    xyzview = py3Dmol.view(width=200,height=200)
    xyzview.addModel(xyz,'xyz')
    xyzview.setStyle({style:{'radius':radius}})
    xyzview.setBackgroundColor('0xeeeeee')
    xyzview.animate({'loop': 'backAndForth'})
    xyzview.zoomTo()
    xyzview.show()
    xyztext = Textarea(value=xyz,layout={'height': '200px','width':'400px'})
    display(xyztext)
    return

aW50ZXJhY3RpdmUoY2hpbGRyZW49KFRleHRhcmVhKHZhbHVlPXUnTycsIGRlc2NyaXB0aW9uPXUnbW9sJywgbGF5b3V0PUxheW91dChoZWlnaHQ9dScxMDBweCcsIHdpZHRoPXUnNDAwcHgnKSnigKY=


In [75]:
@interact(mol=Textarea(value='O',layout={'height': '50px','width':'300px'},description='identifier'),
          basis='vtools/631g',niter=BoundedIntText(min=0,value=1,description='Niter'),
         optimization=['very tight','tight','default','coarse','very coarse'],fullbasis=False, allatoms=False)
def interactive_input_generator(mol,basis, niter=1, optimization='default',fullbasis=False, allatoms=False):
    mol = ob.get_mol(mol,make3D=True)
    xyz = ob.get_xyz(mol)
    info  = 'Molecular formula: {}\n'.format(ob.get_formula(mol))
    info += 'Multiplicity: {}\n'.format(ob.get_multiplicity(mol))
    info += 'Number of electrons: {}\n'.format(ob.get_nelectron(mol))
    info += 'Number of determinants: {}\n'.format(vt.get_number_of_determinants(mol))
    info += 'xyz:\n {}\n'.format(xyz)
    try:
        import py3Dmol
        xyzview = py3Dmol.view(width=100,height=100)
        xyzview.addModel(xyz,'xyz')
        xyzview.setStyle({'stick':{'radius':0.5}})
        xyzview.setBackgroundColor('0xeeeeee')
        xyzview.animate({'loop': 'backAndForth'})
        xyzview.zoomTo()
        molview = xyzview.show()
    except Exception as e:
        pass
    w1 = Textarea(value=info,description='info',layout={'height': '100px','width':'400px'})    
    display(w1)
    orb = vt.get_guess_orbital(mol,basis,fullbasis=fullbasis, allatoms=allatoms)
    if optimization == 'very tight':
        opt = '20 20 20 10 10'
    elif optimization == 'tight':
        opt = '12 12 12 10 10'
    elif optimization == 'coarse':
        opt = '5 5 5 0 0'
    elif optimization == 'very coarse':
        opt = '1 1 1 0 0'
    else:
        opt = '8 8 6 10 10'
    opt = opt + ' ' + str(niter)
    inp = vt.get_valence_input(mol,basis,guess=orb,fullbasis=fullbasis,allatoms=allatoms,opt=opt)
    w2 = Textarea(value=inp,layout={'height': '500px','width':'600px'},description='input')
    display(w2)
    return 

aW50ZXJhY3RpdmUoY2hpbGRyZW49KFRleHRhcmVhKHZhbHVlPXUnTycsIGRlc2NyaXB0aW9uPXUnaWRlbnRpZmllcicsIGxheW91dD1MYXlvdXQoaGVpZ2h0PXUnNTBweCcsIHdpZHRoPXUnMzDigKY=
